# **Stress detection using Decision Tree Classifier algo over HRV data**

In [ ]:
# Using SWELL dataset from Kaggle to predict the mental state of the user using Heart Rate Variability

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Perceptron

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
import tensorflow_hub as hub

import webbrowser

from xgboost import XGBClassifier

In [ ]:
train = pd.read_csv("../input/swell-heart-rate-variability-hrv/hrv dataset/data/final/train.csv")
test = pd.read_csv("../input/swell-heart-rate-variability-hrv/hrv dataset/data/final/test.csv")
train.head()

In [ ]:
conditions = dict(train['condition'].value_counts())
labels = list(conditions.keys())
counts = list(conditions.values())
plt.bar(labels,counts, color ='green',width = 0.4)

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(train['condition'])
train['condition'] = le.transform(train['condition'])
e = test['condition'].unique()
test['condition'] = le.transform(test['condition'])

In [ ]:
f = le.transform(e)
print(f)

In [ ]:
g = le.inverse_transform(f)
print(g)
# 1 = no stress
# 2 = time pressure
# 0 = interruption

In [ ]:
plt.figure(figsize=(12,10))
corr = train.corr()
sns.heatmap(corr, annot=False, cmap=plt.cm.Reds)
plt.show()

In [ ]:
def feature_selection(correlation,threshold):
    selected_features = []
    for i in range(corr.shape[0]):
      if corr.iloc[i,35] > threshold:
        selected_features.append(train.iloc[:,i])
    return pd.DataFrame(selected_features).T

In [ ]:
train.head()

In [ ]:
CORRELATION_THRESHOLD = 0.1
nn_train = train.copy()
reduced_train = feature_selection(corr,CORRELATION_THRESHOLD)
reduced_train.head()

In [ ]:
fig, axarr = plt.subplots(2, 2, figsize=(15, 9))
reduced_train['MEAN_RR'][:70].plot(ax=axarr[0][0])
reduced_train['MEDIAN_RR'][:70].plot(ax=axarr[1][0])
reduced_train['MEDIAN_REL_RR'][:70].plot(ax=axarr[0][1])
reduced_train['SDRR_RMSSD_REL_RR'][:70].plot(ax=axarr[1][1])
axarr[0][0].set_title("Mean_RR")
axarr[0][1].set_title("Median_RR")
axarr[1][0].set_title("Median_Rel_RR")
axarr[1][1].set_title("SDRR_RMSSD_REL_RR")
plt.subplots_adjust(hspace=.3)

In [ ]:
reduced_train[['MEAN_RR','MEDIAN_RR']][:70].plot()

In [ ]:
x_train = reduced_train.iloc[:,:-1]
y_train = reduced_train.iloc[:,-1]

In [ ]:
x_test = test[x_train.columns]
y_test = test['condition']

In [ ]:
dt_model = DecisionTreeClassifier(criterion="entropy", max_depth=13)
dt_model = dt_model.fit(x_train,y_train)

pickle.dump(dt_model,open('model_1.pkl','wb'))
dt_model=pickle.load(open('model_1.pkl','rb'))

# hypertuning
dt_model2 = DecisionTreeClassifier(criterion="entropy", max_depth=14)
dt_model2 = dt_model2.fit(x_train,y_train)

pickle.dump(dt_model2,open('model_2.pkl','wb'))
dt_model2=pickle.load(open('model_2.pkl','rb'))

In [ ]:
dt_prediction = dt_model.predict(x_test)

**Decision Tree Algo report**

In [ ]:
print(classification_report(y_test,dt_prediction))

# **Questionaire based stress level recognition using NLP**
[**Google Colab Notebook**](https://colab.research.google.com/drive/1sNtwXnKt3jyc-h434Gnt38lcXZw_hbaW?usp=sharinghttps://colab.research.google.com/drive/1sNtwXnKt3jyc-h434Gnt38lcXZw_hbaW?usp=sharing)

In [ ]:
def preprocess(file):
    path = '/kaggle/input/npl-data/'+file+'.txt'
    data = pd.read_csv(path, sep = ';')
    hos = []
    for i in range(len(data.emotion)):
        if data['emotion'][i] in ['joy', 'love', 'surprise']:
            hos.append(1) # happy is 1
        else:
            hos.append(0) # sad is 0
    data['hos'] = hos
    return data

In [ ]:
train_data = preprocess('train')
train = train_data.copy()

In [ ]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [ ]:
val = preprocess('val')

In [ ]:
history = model.fit(train.text,
                    train.hos,
                    epochs=40,
                    batch_size=512,
                    validation_data=(val.text, val.hos),
                    verbose = 0)
test = preprocess('test')
results = model.evaluate(test['text'], test['hos'])

print(results)

In [ ]:
predstr = model.predict(train.text)

def postprocessor(preds):
  range = predstr.max()-predstr.min()
  norm_preds = []
  probab = []
  for i in preds:
    norm_preds.append((i - predstr.min()) / range)
    probab.append((i - predstr.min()) * 100 / range)
  return np.mean(probab)

In [ ]:
answers = []
answers.append(input('How would you describe your experience at your workplace/college/school in the past few days?  '))
answers.append(input('How do you like to spend your leisure time? How do you feel after it?  '))
answers.append(input('Life has its ups and downs. Although handling successes can be difficult, setbacks can affect mental health strongly. How do you manage your emotions after failures?  '))
answers.append(input('Any recent pleasant experience that you would like to share?  '))
answers.append(input('In a broad sense, how would you describe the way your life is going on?  '))
results = model.predict(answers)
final_state = postprocessor(results)
print('Your mental health score is:', final_state)

In [ ]:
if final_state<25:
    print("It seems that your stress level might have been high momentarily due to some intense activity but I assure you that it wasn't any mental health issue")
else:
    url = "https://www.geeksforgeeks.org"
    webbrowser.open(url)